In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import matplotlib
matplotlib.pyplot.style.use('seaborn')
matplotlib.rcParams['figure.figsize'] = (15, 5)

%matplotlib inline

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
np.set_printoptions(precision=2, suppress=True)

In [ ]:
import math
import copy

import scipy.stats as stats

In [ ]:
from sklearn import model_selection, metrics, datasets

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Model

# $Model(x_i, w) = w_0 + \sum_j^d w_j x_i^j$

$x_i$ - $i'th$ featrue in matrix of features $X$

$x_i^j$ - $j'th$ featrue value of current feature $i$

$w_j$ - weight for $x_i^j$ feature value

$w_0$ - free coeficent

---

# In matrix form:

In matrix (add $x_i^0$ and set it to $1$, and add $w_0$ for it)

$Model(X, w) = Xw$

$\begin{bmatrix}
(x_0^0=1), & ..., & (x_0^j) \\
(x_1^0=1), & ..., & (x_1^j) \\
... \\
(x_n^0=1), & ..., & (x_n^j) \\
\end{bmatrix}$
$\begin{bmatrix}
w_0 \\
...\\
w_j
\end{bmatrix}$

In [ ]:
X = np.array([
    [1, 2, 5],
    [1, 5, 5],
    [1, 8, 5],
], dtype=np.float64)

w = np.array([0.1, 0.1, 0.1], dtype=np.float64)

yhat = X.dot(w)
yhat

##### $y = Xw + \epsilon$

$\begin{bmatrix}
y_0 \\
... \\
y_n \\
\end{bmatrix} = $
$\begin{bmatrix}
x_0^0 & x_0^1 &  ... & x_0^j \\
... \\
x_n^0 & x_n^1 &  ... & x_n^j
\end{bmatrix}$
$\begin{bmatrix}
w_0 \\
...\\
w_j
\end{bmatrix} +$
$\begin{bmatrix}
\epsilon_0 \\
...\\
\epsilon_n
\end{bmatrix}$

# Cost function

$Cost(w, X, y) = \frac{1}{n} \sum_i \left(Model(x_i, w) - y_i \right)^2$

$Cost(w, X, y) = \frac{1}{n}\left\| Xw-y \right\|^2$

$\triangledown Cost = \frac{2}{n} X^{T}(Xw - y)$

---


### _Intuition for cost function derivative: (Not a strong proof!)_:

$Cost(w, X, y) = \frac{1}{n} \left\|Xw - y\right\|^2$

$\frac{d}{dw}(Xw - y)^2$

$2(Xw - y)\frac{d}{dw}\left[ Xw - y \right]$

$2(Xw - y)(X)$

$\frac{2}{n} X^T(Xw - y)$

In [ ]:
X = np.array([
    [1, 2, 15,  8],
    [1, 5, 45, 12],
    [1, 8, 53, 33],
], dtype=np.float64)

y = np.array([9, 21, 33], dtype=np.float64)
w = np.array([0.1, 0.1, 0.1, 0.1], dtype=np.float64)

cost = np.linalg.norm(X.dot(w) - y)**2 / len(X)
cost

# Same as above:
# sum((X.dot(w) - y)**2) / len(X)

grad = (2/len(X)) * X.T.dot(X.dot(w) - y)
grad

# Task - Cost function minimization

$Cost(w, X, y) \Rightarrow \underset{w}{min}$

# Gradient Descent

$w^t = w^{t-1} - \alpha_t \triangledown Cost(w^{t-1},X,y)$

$\triangledown$ - gradient _(vector of derivatives)_

$\alpha_t$ - learning rate for current step

$t$ - iteration

##### Stop: $\left\| w_t - w^{t-1} \right\| < \epsilon$
_если разница двух последовательных приближений не слишком велика_

##### Vector of derivatives (Gradient):

Use chain rule:

$\frac{\partial }{\partial w_0} Cost =
\frac{1}{n} \sum_i 2((w_0 + w_1 \cdot x_i^{(1)} + w_2 \cdot x_i^{(2)} + ... + w_j \cdot x_i^{(j)}) - y_i) \cdot 1
$

$\frac{\partial }{\partial w_1} Cost =
\frac{1}{n} \sum_i 2((w_0 + w_1 \cdot x_i^{(1)} + w_2 \cdot x_i^{(2)} + ... + w_j \cdot x_i^{(j)}) - y_i) \cdot x_i^{(1)}
$

...

$\frac{\partial }{\partial w_j} Cost =
\frac{1}{n} \sum_i 2((w_0 + w_1 \cdot x_i^{(1)} + w_2 \cdot x_i^{(2)} + ... + w_j \cdot x_i^{(j)}) - y_i) \cdot x_i^{(j)}
$

---

##### Vectors of derivatives (Gradient):

$\triangledown Cost = \begin{bmatrix}
\frac{2}{n} \sum_i ((w_0 + w_1 \cdot x_i^{(1)} + w_2 \cdot x_i^{(2)} + ... + w_j \cdot x_i^{(j)}) - y_i) \cdot 1 \\
\frac{2}{n} \sum_i ((w_0 + w_1 \cdot x_i^{(1)} + w_2 \cdot x_i^{(2)} + ... + w_j \cdot x_i^{(j)}) - y_i) \cdot x_i^{(1)} \\
... \\
\frac{2}{n} \sum_i ((w_0 + w_1 \cdot x_i^{(1)} + w_2 \cdot x_i^{(2)} + ... + w_j \cdot x_i^{(j)}) - y_i) \cdot x_i^{(j)}
\end{bmatrix}$

---

##### Vectors of derivatives (Gradient):

$\triangledown Cost = \begin{bmatrix}
\frac{2}{n} (x_0w - y) \cdot 1 \\
\frac{2}{n} (x_iw - y) \cdot x^{(1)}_i \\
... \\
\frac{2}{n} (x_nw - y) \cdot x^{(j)}_n
\end{bmatrix}$

---

##### Vectors of derivatives (Gradient):
$\triangledown Cost = \frac{2}{n} X^{T}(Xw - y)$

$X^T$ _is for_ "$\cdot x^{(j)}$" on all $x_i$, see above

In [ ]:
def mse(w, X, y):
    return np.linalg.norm(X.dot(w) - y)**2 / len(X)

In [ ]:
def gmse(w, X, y):
    return (2/len(X)) * X.T.dot(X.dot(w) - y)

# Regularization 

## Ridge

$Cost(w, X, y) = Cost(w, X, y) + \lambda \left\| w \right\|_2^2 \Rightarrow \underset{w}{min}$

$Cost(w, X, y) \Rightarrow \underset{w}{min}$

$Cost(w, X, y) = (Xw - y)^T(Xw - y) + \lambda \left\| w \right\|_2^2 =
(Xw - y)^T(Xw - y) + \lambda w^Tw
$

$\triangledown Cost(w, X, y) =
\triangledown \left[ (Xw - y)^T(Xw - y) + \lambda w^Tw \right] =
\triangledown \left[ (Xw - y)^T(Xw - y) \right] + \triangledown  \left[ \lambda w^Tw \right]
$

$\triangledown Cost(w, X, y) = 2X^T(Xw - y) + 2\lambda w$

In [ ]:
def ridge(w, l):
    w = w.copy()
    w[0] = 0 # Don’t penalize intercept term w0
    return 2 * l * w

# . . .

$w^t = w^{t-1} - \alpha_t \triangledown Cost(w^{t-1},X,y)$

$w^t = w^{t-1} - \alpha_t \cdot (2X^T(Xw^{t-1} - y) + 2\lambda w)$

In [ ]:
def minimize(X, y, cost, grad, reg, iterations, epsilon, alpha, reg_coef):
    # add coef of ones
    X = np.append(np.ones(len(X[0])), X.T).reshape(4,3).T

    # initilize weights vector
    w = np.zeros(len(X[0]), dtype=np.float64)

    # parameters
    weights = [w]
    error = []

    for iteration in range(iterations):
        w = w - alpha * (grad(w, X, y) + reg(w, reg_coef))
        if np.linalg.norm(w - weights[-1]) < epsilon:
            break
        weights.append(w)
        error.append(cost(w, X, y))

    return w[0], w[1:], error

In [ ]:
def predict(X, w, coef):
    X = np.append(np.ones(len(X[0])), X.T).reshape(4,3).T
    w = np.append(np.array(coef), w)

    return X.dot(w)

In [ ]:
X = np.array([
    [0.2, 0.15, 0.8],
    [0.5, 0.45, 0.12],
    [0.8, 0.53, 0.33],
], dtype=np.float64)

y = np.array([
    sum(X[0] * 2),
    sum(X[1] * 2),
    sum(X[2] * 2),
], dtype=np.float64)

In [ ]:
y

In [ ]:
coef, w, error = minimize(X, y, mse, gmse, ridge,
                          iterations = 50, epsilon = 0.003, alpha = 0.1, reg_coef = 0.001)
# epsilon = 0.0003, alpha = 0.1, reg_coef = 0.00001
coef

w
predict(X, w, coef) # array([2.3 , 2.14, 3.32])

In [ ]:
plt.plot(error);

# Check analytically

In [ ]:
wp = np.linalg.inv(X.T.dot(X)).dot(X.T.dot(y))

wp
X.dot(wp)

# Check analytically with Ridge regularizator

In [ ]:
L = 0.001*np.identity(len(X))
wp = np.linalg.inv(X.T.dot(X) + L).dot(X.T.dot(y))

wp
X.dot(wp)

# Check by sklearn

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X, y)

reg.intercept_ 

reg.coef_
reg.predict(X)